# Train Model on Large Training Set

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

import music_generator.sequence_utils as sequence_utils
import music_generator.utils as utils
from music_generator.midi_converter import DiscreteTimeMidiConverter
from music_generator.model import MusicModel

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0-dev20200327


# Make sure CUDA and GPU are working

In [2]:
for message in utils.check_cuda_and_gpu():
    print(message)

CUDA and GPU Available...


# Create model

In [3]:
description = 'paper_hparams_384classes_64embed_128batch_adam'

model = MusicModel(
    n_classes=384,
    embed_dims=64,
    rnn_size=256,
    rnn_layers=3,
    dense_size=None,
    dense_layers=1,
    dropout_rate=0.3,
    batch_norm=True,
    init_lr=None,
    dense_activation='relu',
    ckpt_dir='./training_checkpoints/{}'.format(description),
    log_dir='./logs/{}'.format(description)
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          24576     
_________________________________________________________________
lstm (LSTM)                  (None, None, 256)         328704    
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 256)         1024      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256)         1

# Create Dataset from Bach Chorales

In [4]:
data_path = './bach_chorales_midi/chorale_set_full/'

midi_converter = DiscreteTimeMidiConverter()
sequences = midi_converter.convert_folder(data_path)
sequences, labels = sequence_utils.window(sequences, window_size=128)

x_train, x_val, y_train, y_val = train_test_split(sequences, labels, test_size=0.1, shuffle=True)
print('Training Sequences: {}\nValidation Sequences: {}'.format(len(x_train), len(x_val)))

batch_size = 128

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))

dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
dataset_val = dataset_val.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

Training Sequences: 283227
Validation Sequences: 31470


# Train Model on Dataset

In [5]:
epochs = 100
history = model.fit(dataset,
                    val_data=dataset_val,
                    epochs=epochs,
                    verbose=1)

Epoch 1/100
2212/2212 [==============================] - 171s 77ms/step - loss: 1.6060 - sparse_categorical_accuracy: 0.5477 - val_loss: 1.0028 - val_sparse_categorical_accuracy: 0.6814
Epoch 2/100
2212/2212 [==============================] - 170s 77ms/step - loss: 1.0274 - sparse_categorical_accuracy: 0.6761 - val_loss: 0.8437 - val_sparse_categorical_accuracy: 0.7264
Epoch 3/100
2212/2212 [==============================] - 170s 77ms/step - loss: 0.9005 - sparse_categorical_accuracy: 0.7116 - val_loss: 0.7697 - val_sparse_categorical_accuracy: 0.7500
Epoch 4/100
2212/2212 [==============================] - 171s 77ms/step - loss: 0.8274 - sparse_categorical_accuracy: 0.7316 - val_loss: 0.7157 - val_sparse_categorical_accuracy: 0.7656
Epoch 5/100
2212/2212 [==============================] - 171s 78ms/step - loss: 0.7830 - sparse_categorical_accuracy: 0.7444 - val_loss: 0.6962 - val_sparse_categorical_accuracy: 0.7724
Epoch 6/100
2212/2212 [==============================] - 172s 78ms/ste

# Sample from model

In [6]:
# sample 5 midi sequences, of length 1000 events, from the model with a middle-C note as the seed
for i in range(5):
    seed = [60]
    generated_sequence = model.generate_sequence(1000, seed)
    midi_converter.convert_to_file(generated_sequence, './generated files/{}'.format(description), 'sample{}.mid'.format(i))